In [2]:
import smart_open
from typing import List, Set, Tuple
import pandas as pd

import numpy as np

from elasticsearch import Elasticsearch
from elasticsearch.connection import create_ssl_context

In [10]:
from onai.ml.peers.experiment.ranknet_gen import RankNetCandidateSuggestion
from onai.ml.peers.suggestions.bert import IndustryPredictor

In [11]:
tst = RankNetCandidateSuggestion("host.docker.internal",
                                 "9200",
                                 f"{model_path}/model/torch_net.pkl",
                                 cols,
                                 industry_predictor=IndustryPredictor()
                                )

Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'onai.ml.peers.experiment.ranknet_candidate_gen.RankNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [ ]:
tst.client

In [6]:
model_path = "s3://oaknorth-ml-dev-eu-west-1/andrei/peers/ltr"

with smart_open.open(f"{model_path}/Base_borrowers.xlsx", "rb") as f:
    annotated_companies = pd.read_excel(f)

annotated_companies = annotated_companies.apply(map_get_currency, axis=1)
annotated_companies = annotated_companies.rename(columns = {
    "Case": "task_id",
    "Base Borrower Name": "company_name",
    "Base Borrower Description": "business_description",
    "Base Borrower Region": "region",
    "Base Borrower Sector": "sector_/_or__sic_codes",
    "Base Borrower Revenue Range": "revenue_range_pretty"
})

NameError: name 'map_get_currency' is not defined

In [7]:
features = []
for _,borrower_raw in annotated_companies[annotated_companies['task_id'] == 43].iterrows():
    borrower = company_details_from_pandas(borrower_raw)
    
    es_results = feature_extractor.get_es_candidates(borrower, size=20)
    candidates_with_features = feature_extractor.extract_features_for_suggestions(borrower, es_results)
    for el in candidates_with_features:
        el["task_id"] = borrower_raw["task_id"]
    features.extend(candidates_with_features)

abeb6e5d-a0cb-5161-ad03-36e793426658 has no fye
065fb0a1-e3ff-57ef-ab45-58a6433e3060 has no fye
2317e7f3-42ba-531b-b655-05e797b2f0e2 has no fye
1a0def80-d2db-56e9-a112-1b253fd0627b has no fye
201f9f18-0069-53aa-82a4-b86853fe37ad has no fye
98325ceb-fc23-5b49-9c8e-157b4490b3a7 has no fye
9d87e970-7284-5a0f-9234-28a2914a67f6 has no fye
133b7820-5a38-5364-99c2-8d511e7de3c0 has no fye


In [8]:
features_df = pd.DataFrame(features)

In [9]:
features_df

,base_name,peer_name,country_overlap,weighted_symmetric_diff,weighted_intersection_negative_sample,weighted_intersection_negative_sample_tail_end,weighted_intersection,peer_diff,bert_embedding_sim_hidden_state,bert_embedding_sim,bert_embedding_1st_sim,bert_embedding_pretrained,description_length,es_score_normed,es_score,predicted_industries_overlap,revenue_ratio,task_id
0,"Jerash Holdings (US), Inc.","Infinity Headwear and Apparel, Inc.",1,5.441960,0.037775,0.053509,0.079640,7.334767,0.827623,0.954878,0.890374,0.894746,207,1.000000,97.161650,3,119.714594,43
1,"Jerash Holdings (US), Inc.","Salt Life, LLC",1,5.553958,0.053231,0.142641,0.146386,7.487807,0.849651,0.968385,0.884470,0.903487,595,0.918154,89.209400,2,0.000000,43
2,"Jerash Holdings (US), Inc.","Headmaster, Inc.",1,5.864277,0.057018,0.148773,0.140218,7.053312,0.645526,0.961189,0.816289,0.642288,315,0.863166,83.866600,1,0.000000,43
3,"Jerash Holdings (US), Inc.","National Cap And Sportswear, Inc.",1,4.874842,0.000000,0.015928,0.015800,5.882731,0.426715,0.932428,0.763605,0.252216,103,0.855514,83.123160,2,19.198276,43
4,"Jerash Holdings (US), Inc.",Bee Sales Company,1,5.277227,0.026921,0.036734,0.034770,7.114171,0.467416,0.947715,0.765168,0.345347,396,0.809621,78.664085,1,0.000000,43
5,"Jerash Holdings (US), Inc.","Burma Bibas, LLC",1,5.636325,0.146377,0.266744,0.232735,5.934822,0.855472,0.970515,0.750630,0.885216,302,0.805830,78.295780,1,0.000000,43
6,"Jerash Holdings (US), Inc.","Game Sportswear, LTD",1,5.663484,0.097245,0.193538,0.184543,7.682722,0.852203,0.970317,0.903148,0.893835,447,0.780590,75.843370,2,0.000000,43
7,"Jerash Holdings (US), Inc.",Global Apparel Corp.,1,6.072843,0.103776,0.131924,0.134923,7.470420,0.895273,0.965610,0.823776,0.912758,1136,0.764490,74.279130,1,0.000000,43
8,"Jerash Holdings (US), Inc.",Babtech Inc.,1,4.565952,0.125174,0.125174,0.092108,7.030503,0.691628,0.932371,0.755752,0.740644,123,0.727285,70.664170,1,0.000000,43
9,"Jerash Holdings (US), Inc.",Knitcraft Corporation,1,5.895037,0.028591,0.104375,0.117072,6.070674,0.837194,0.967403,0.818231,0.890111,350,0.716657,69.631530,2,0.687132,43


## Implement Ranker ABC

In [10]:
from typing import List

import torch
import torch.nn as nn

from onai.ml.peers.experiment.candidate_gen import ESCandidateSuggestion

In [11]:
from onai.ml.peers.experiment.annotation_generator import (
    CompanyDetail,
    PeerSuggestion,
    convert_monetary_financial_to_value,
    fetch_financials_from_dp,
    last_year_total_revenue,
)

In [12]:
class RankNet(nn.Module):
    def __init__(self, x_dim):
        super(RankNet, self).__init__()
        self.fc = nn.Linear(x_dim, 1, bias=False)
    
    def forward(self, x):
        batch_size, d = x.shape
        instance_dim = d//2
        peer1 = x[:, :instance_dim]
        peer2 = x[:, instance_dim:]
        o = self.fc(peer1) - self.fc(peer2)

        return o

    def predict(self, x):
        return self.fc(x)

In [8]:
cols=["es_score",
      "es_score_normed",
      "predicted_industries_overlap",
      "description_length",
      "country_overlap",
      "weighted_symmetric_diff",
      "weighted_intersection",
      "weighted_intersection_negative_sample",
      "bert_embedding_sim_hidden_state",
      "bert_embedding_sim",
      "bert_embedding_1st_sim",
      "bert_embedding_pretrained",
      "weighted_intersection_negative_sample_tail_end",
      "peer_diff",
      "revenue_ratio"
]

In [14]:
class RanknetWrapper:
    def __init__(self, model_path:str, cols: List[str]):
        self.cols = cols
        with smart_open.open(model_path, 'rb') as f:
            self.rank_net = torch.load(f)
    
    def predict_score(self, X_df: pd.DataFrame) -> np.array:
        net_input = torch.from_numpy(X_df[cols].to_numpy()).float()
        return self.rank_net.predict(net_input).detach().numpy()

class RankNetCandidateSuggestion(ESCandidateSuggestion):
    def __init__(self, host:str, port:str, ranknet_model_path:str, feature_cols, *args, **kwargs):
        super(RankNetCandidateSuggestion, self).__init__(host, port, *args, **kwargs)
        self.feature_extractor = PeerFeatureExtractor(EmbeddingRetriever(), self.peer_es_client)
        self.ranknet_wrapper = RanknetWrapper(ranknet_model_path, feature_cols)
        
    def _extract_peers_with_features(self, 
                                     base_company: CompanyDetail, 
                                     size: int = 100
                                    ) -> Tuple[List[dict], pd.DataFrame]:
        es_results = feature_extractor.get_es_candidates(borrower, size)
        candidates_with_features = feature_extractor.extract_features_for_suggestions(borrower, es_results)
        return es_results, pd.DataFrame(candidates_with_features)
    
    def suggest_candidates(self, base_company: CompanyDetail, size=100) -> List[PeerSuggestion]:
        generated_candidates, features_for_candidates = self._extract_peers_with_features(base_company, size)
        scores = self.ranknet_wrapper.predict_score(features_for_candidates)
        
        ret = []
        for rank, (score, row) in enumerate(sorted(zip(scores, generated_candidates), reverse=True)):
            cd = row.detail
            print(score, cd)
        return []

In [15]:
tst = RankNetCandidateSuggestion("host.docker.internal", "9200", f"{model_path}/model/torch_net.pkl", cols)

Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [16]:
features = []
for _,borrower_raw in annotated_companies[annotated_companies['task_id'] == 8].iterrows():
    borrower = company_details_from_pandas(borrower_raw)
    res = tst.suggest_candidates(borrower, size=20)

32617fbe-8615-50cf-8a4f-3665fae93e3e has no fye
2d5b3e9f-710f-5d62-96fc-952cf5c99314 has no fye
06de0044-a75c-5880-8875-79110eca3c64 has no fye
0943446c-afc7-5cc4-9eae-3131274d138a has no fye
09f05de2-295d-5149-a9ac-30740b806cab has no fye


[17.347778] CompanyDetail(name='Premium Food Restaurants S.A.', description='Premium Food Restaurants S.A. operates Japanese restaurants in Poland. Its restaurants offer sushi and Japanese culture foods. The company is headquartered in Warsaw, Poland. As of August 7, 2018, Premium Food Restaurants S.A. was taken private.', region='Europe', fye=datetime.date(2008, 12, 31), financials=defaultdict(<class 'list'>, {'EBITDA': [Financial(val=0.1199866412868533, year=2008, currency='EUR', magnitude='m'), Financial(val=-0.0008073241823416212, year=2009, currency='EUR', magnitude='m'), Financial(val=0.05327385944701148, year=2010, currency='EUR', magnitude='m'), Financial(val=0.4071306949341979, year=2011, currency='EUR', magnitude='m'), Financial(val=0.42067296431188084, year=2012, currency='EUR', magnitude='m'), Financial(val=0.2120043152732575, year=2013, currency='EUR', magnitude='m'), Financial(val=-0.034760001211135844, year=2014, currency='EUR', magnitude='m'), Financial(val=-0.472871979

In [17]:
res

[]

In [17]:
with smart_open.open(f"{model_path}/query_document_features.csv", 'r') as f:
    rld =  pd.read_csv(f)

In [19]:
rld[rld['task_id'] == 8]

,task_id,base_name,peer_name,country_overlap,weighted_symmetric_diff,weighted_intersection_negative_sample,weighted_intersection_negative_sample_tail_end,weighted_intersection,peer_diff,bert_embedding_sim_hidden_state,...,icr_ebit,debt_ebitda,net_debt_ebitda,capex_rev,icr_ebitda,roa,roe,ebit_marg,primary_naics_node,primary_naics_node_desc
140,8,Alpin Gastronomie GmbH ML Test,Trattoria La Napoletana Srl,0,6.787283,0.029736,0.029736,0.023892,11.021471,0.505233,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,8,Alpin Gastronomie GmbH ML Test,Monticelli Srl,0,6.826824,0.027371,0.027371,0.022341,9.029406,0.427599,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,8,Alpin Gastronomie GmbH ML Test,Wienerwald Restaurants GmbH,1,6.891107,0.000000,0.074106,0.079466,4.449886,0.771147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,8,Alpin Gastronomie GmbH ML Test,MAREDO Gaststätten-GmbH & Co Betriebs KG,0,7.120907,0.000000,0.036734,0.049023,5.590750,0.796352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,8,Alpin Gastronomie GmbH ML Test,DINEA Gastronomie GmbH,0,6.963060,0.000000,0.050435,0.060132,5.148874,0.731255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,8,Alpin Gastronomie GmbH ML Test,Diedamskopf Alpin Tourismus GmbH & CO KG,1,6.935861,0.000000,0.046705,0.056155,7.971183,0.580716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,8,Alpin Gastronomie GmbH ML Test,BLOCK HOUSE Restaurantbetriebe AG,0,6.807002,0.011224,0.040570,0.054149,6.161230,0.809366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,8,Alpin Gastronomie GmbH ML Test,C. Wöllhaf Gastro Service GmbH,0,6.740365,0.000000,0.025578,0.033004,5.894371,0.721281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,8,Alpin Gastronomie GmbH ML Test,SSP Airport Gastronomiegesellschaft mbH,0,7.021586,0.005413,0.021754,0.029471,8.090746,0.764459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,8,Alpin Gastronomie GmbH ML Test,Vapiano SE,0,6.650370,0.029731,0.039800,0.050126,5.780054,0.861751,...,-6.632679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
